In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# MES AÑO DE CONCILIACIÓN

In [ ]:
month_conc = 'August'
year_conc = '2022'

# RUTAS DE ACCESO

### SWAP DEL DIA 

In [ ]:
allfiles_disp_trad = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2022\08. AGOSTO\ARCHIVO DISPERSIONES\Dispersión*.xlsx', recursive=False)
allfiles_disp_trad

### FAST PAYMENTS

In [ ]:
allfiles_fp = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2022\08. AGOSTO\ARCHIVO DISPERSIONES FP\Dispersión-FP*.xlsx',recursive=False)
allfiles_fp

# SWAP DEL DIA

In [ ]:
swap_dia_total = pd.concat([pd.read_excel(f,sheet_name = 'Swap del día')
                            .assign(file_name=os.path.basename(f)) for f in allfiles_disp_trad])
swap_dia_total['Date'] = swap_dia_total["file_name"].str[-13:-5]
swap_dia_total['Date']= pd.to_datetime(swap_dia_total['Date'], format='%Y%m%d').dt.date
swap_dia_total['Date']= pd.to_datetime(swap_dia_total['Date'])
swap_dia_total.sort_values(by=['Date'],ascending=True,inplace=True)
swap_dia_total['report_date_local']= pd.to_datetime(swap_dia_total['report_date_local'])
swap_dia_total['month_conc'] = month_conc
swap_dia_total['year_conc'] = year_conc
swap_dia_total['month_report'] = swap_dia_total['report_date_local'].dt.strftime('%B')
swap_dia_total['year_report'] = swap_dia_total['report_date_local'].dt.year.astype(str)
swap_dia_total.head()

In [ ]:
swap_dia_total.shape

In [ ]:
swap_dia_total.dtypes

In [ ]:
pivot_total = pd.pivot_table(swap_dia_total,index=['Date'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_total.reset_index(drop=False, inplace=True)
pivot_total.head()

In [ ]:
pivot_parcial = pd.pivot_table(swap_dia_total,index=['tipo','Date'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_parcial.reset_index(drop=False, inplace=True)
pivot_parcial.head()

## CATEGORIZACIÓN DE REPROGRAMACIONES

In [ ]:
swap_dia_total_repro = swap_dia_total[(swap_dia_total['tipo'] == 'Reprogramación')]
swap_dia_total_repro.head()

In [ ]:
swap_dia_total_repro.shape

In [ ]:
swap_dia_total_repro.loc[(swap_dia_total_repro['tipo'] == 'Reprogramación')
                   & (swap_dia_total_repro['month_report'] == month_conc) 
                   & (swap_dia_total_repro['year_report'] == year_conc),'tipo_reprogramacion'] = 'repro_mismo_mes'
swap_dia_total_repro['tipo_reprogramacion'].fillna('repro_otro_mes',inplace=True)
swap_dia_total_repro.head()

In [ ]:
pivot_tipo_repro = pd.pivot_table(swap_dia_total_repro,index=['Date'],
                                  columns=['tipo_reprogramacion'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_tipo_repro.reset_index(drop=False, inplace=True)
pivot_tipo_repro.head()

# FAST PAYMENTS

In [ ]:
fp_dia_total = pd.concat([pd.read_excel(f,sheet_name='FAST PAYMENTS')
                          .assign(file_name=os.path.basename(f)) for f in allfiles_fp])
fp_dia_total['Date'] = fp_dia_total["file_name"].str[-15:-5]
fp_dia_total['Date']= pd.to_datetime(fp_dia_total['Date'])
fp_dia_total['report_date_local']= pd.to_datetime(fp_dia_total['report_date_local'])
fp_dia_total['month_conc'] = month_conc
fp_dia_total['year_conc'] = year_conc
fp_dia_total['month_report'] = fp_dia_total['report_date_local'].dt.strftime('%B')
fp_dia_total['year_report'] = fp_dia_total['report_date_local'].dt.year.astype(str)
fp_dia_total.head()

In [ ]:
fp_dia_total.shape

In [ ]:
fp_dia_total.dtypes

In [ ]:
pivot_fp_total = pd.pivot_table(fp_dia_total,index=['Date'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_fp_total.reset_index(drop=False, inplace=True)
pivot_fp_total.head()

In [ ]:
pivot_fp_parcial = pd.pivot_table(fp_dia_total,index=['tipo','Date'],values=['total_due'],aggfunc = ['sum', len],margins=True)
pivot_fp_parcial.reset_index(drop=False, inplace=True)
pivot_fp_parcial.head()

## CATEGORIZACIÓN DE REPROGRAMACIONES - FAST PAYMENTS

In [ ]:
fp_dia_total_repro = fp_dia_total[(fp_dia_total['tipo'] == 'Reprogramación')]
fp_dia_total_repro.head()

In [ ]:
fp_dia_total_repro.shape

In [ ]:
fp_dia_total_repro.loc[(fp_dia_total_repro['tipo'] == 'Reprogramación')
                   & (fp_dia_total_repro['month_report'] == month_conc) 
                   & (fp_dia_total_repro['year_report'] == year_conc),'tipo_reprogramacion'] = 'repro_mismo_mes'
fp_dia_total_repro['tipo_reprogramacion'].fillna('repro_otro_mes',inplace=True)
fp_dia_total_repro.head()

In [ ]:
pivot_tipo_repro_fp = pd.pivot_table(fp_dia_total_repro,index=['Date'],
                                  columns=['tipo_reprogramacion'],
                                  values=['total_due']
                                  ,aggfunc = ['sum', len],margins=True)
pivot_tipo_repro_fp.reset_index(drop=False, inplace=True)
pivot_tipo_repro_fp.head()

# UNION REPROGRAMACIONES

In [ ]:
union_repros = pd.concat([swap_dia_total_repro,fp_dia_total_repro])
union_repros.head()

In [ ]:
union_repros.shape

# CUBETA ERROR

In [ ]:
excel_error = pd.read_excel(r'C:\Users\Cristian.Aparicio\Documents\CUBETAS\2022\07. JULIO\CUBETA ERROR JULIO 2022.xlsx',sheet_name='Cubeta ErrorLastMonth',converters={'report_id':str})
excel_error.head()

In [ ]:
excel_error.shape

# CRUCE 1:  REPROGRAMACIONES VS CUBETA EN ERROR

In [ ]:
cruce_1 = union_repros.merge(excel_error[['report_id','status_cubeta','esquema final']] ,on='report_id', how='left')
cruce_1.head()


In [ ]:
cruce_1.shape

# CRUCE_2: CUBETA ERROR VS REPROGRAMACIONES

In [ ]:
cruce_2 = excel_error.merge(union_repros[['report_id','report_date_local','tipo','tipo_reprogramacion',
                                         'file_name','status']] ,on='report_id', how='left')
cruce_2.head()

In [ ]:
cruce_2.shape

# EXCEL FINAL

In [ ]:
writer = pd.ExcelWriter('VALIDACION SALIDA - CUBETAS.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
cruce_1.to_excel(writer, sheet_name='DETALLE_1')
cruce_2.to_excel(writer, sheet_name='DETALLE_2')
writer.close()